In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Practise').getOrCreate()
spark

# Mlib

There are 2 diff technique for pyspark ml

1) RDD

2) Dataftame API----- Most famous





In [4]:
training = spark.read.csv('test3.csv', header=True, inferSchema= True)
training.show()

+---------+---+----------+------+
|     Name|Age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [ ]:
#Experience and age are independent variable

#Salary is dependent variable


In [6]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

#Group allindependent column------ which is known as Vector assembler

[Age, Experience]------> This willbe our new feature


In [10]:
from pyspark.ml.feature import VectorAssembler

featureassembler= VectorAssembler(inputCols=['Age','Experience'], outputCol="Independent Features")

In [11]:
output=featureassembler.transform(training)

In [12]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|Age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [13]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [14]:
finalize_data= output.select(['Independent Features','Salary'])
finalize_data.show()

#Data is ready

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



# Now ml algo

In [21]:
from pyspark.ml.regression import LinearRegression

train_data, test_data= finalize_data.randomSplit([0.75,0.25])

regressor= LinearRegression(featuresCol='Independent Features', labelCol='Salary')

regressor=regressor.fit(train_data)

In [22]:
##Coefficient

regressor.coefficients

DenseVector([-714.2857, 3485.7143])

In [23]:
#Intercept

regressor.intercept

26857.142857139796

In [27]:
# Prediction

pred_result= regressor.evaluate(test_data)
pred_result.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [30.0,8.0]| 25000|33314.28571428435|
|         [31.0,10.0]| 30000|39571.42857142653|
+--------------------+------+-----------------+



In [29]:
pred_result.meanAbsoluteError,pred_result.meanSquaredError

(8942.857142855439, 80369795.91833645)